In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, Flatten
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [2]:
#sliding window sequence builder
def create_sequences(df, feature_cols, target_col, window=20):
    X, y = [], []
    values = df[feature_cols].values
    targets = df[target_col].values

    for i in range(len(df) - window):
        X.append(values[i:i+window])
        y.append(targets[i+window])

    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int32)


In [3]:
def build_cnn_model(n_features, window):
    model = Sequential([
        Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(window, n_features)),
        MaxPooling1D(pool_size=2),
        Conv1D(filters=64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [4]:
import pandas as pd
df = pd.read_csv("E://PHD/Course materials/Sem 1/Data Mining/Assignment/workspace/stock_price_trend_prediction/stock_price_pred_data_mining/Data/05_cluster_output/cluster_output_Agglomerative.csv")
# 1. Compute next-week (5 trading days) forward return per stock
df['next_week_close'] = df.groupby('stock_id')['Close'].shift(-5)
df['next_week_return'] = (df['next_week_close'] - df['Close']) / df['Close']
# 2. Binary target: 1 = uptrend, 0 = downtrend or flat
df["target"] = (df["Close"].shift(-5) > df["Close"]).astype(int)
# 3. Drop rows where we can't compute future return (last 5 days per stock)
df = df.dropna(subset=['next_week_return'])
# Optional: drop helper column
df = df.drop(columns=['next_week_close'])
drop_cols = ['Date', 'stock_id', 'cluster', 'target', 'next_week_return']
feature_cols = [c for c in df.columns if c not in drop_cols]

print("Number of features:", len(feature_cols))
print("Features:", feature_cols)

Number of features: 24
Features: ['Open', 'High', 'Low', 'Close', 'Volume', 'MA7', 'MA21', 'EMA20', 'EMA50', 'RSI_14', 'MACD', 'MACD_signal', 'MACD_hist', 'BB_upper', 'BB_middle', 'BB_lower', 'CCI_14', 'CMF_20', 'Stoch_K', 'Stoch_D', 'Momentum_10', 'Daily_Return', 'Log_Return', 'agg_cluster']


In [ ]:
WINDOW = 20  # typically 20, 30, or 60
cluster_cnn_models = {}
cluster_metrics = {}

for cl in df['agg_cluster'].unique():
    print(f"\nTraining 1D CNN for Cluster {cl}")

    sub = df[df['agg_cluster'] == cl].copy().sort_values('Date')

    # ---- Build sequences
    X, y = create_sequences(sub, feature_cols, 'target', window=WINDOW)

    if len(X) < 200:
        print(f"Cluster {cl}: Not enough samples")
        continue

    # Train/Test split (time-series aware)
    split_idx = int(len(X) * 0.7)
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    # Build model
    model = build_cnn_model(n_features=X.shape[2], window=WINDOW)

    # Train CNN
    es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=64,
        validation_split=0.2,
        callbacks=[es],
        verbose=1
    )

    cluster_cnn_models[cl] = model

    # ---- Predictions
    y_prob = model.predict(X_test).flatten()
    y_pred = (y_prob > 0.5).astype(int)

    # ---- Metrics
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    try:
        auc = roc_auc_score(y_test, y_prob)
    except:
        auc = 0.0

    cluster_metrics[cl] = [acc, prec, rec, f1, auc]

    print(f"""
       Cluster {cl} — 1D CNN Results
       ------------------------------
       Accuracy     : {acc:.4f}
       Precision    : {prec:.4f}
       Recall       : {rec:.4f}
       Macro F1     : {f1:.4f}
       ROC-AUC      : {auc:.4f}
    """)



Training 1D CNN for Cluster 3
Epoch 1/30


c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


361/361 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5072 - loss: 5585.6855 - val_accuracy: 0.5564 - val_loss: 46.2312
Epoch 2/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5224 - loss: 15.0249 - val_accuracy: 0.5651 - val_loss: 3.2252
Epoch 3/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5217 - loss: 1.8685 - val_accuracy: 0.5641 - val_loss: 6.2863
Epoch 4/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5213 - loss: 1.0422 - val_accuracy: 0.5660 - val_loss: 4.9881
Epoch 5/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5214 - loss: 0.6945 - val_accuracy: 0.5658 - val_loss: 5.0072
Epoch 6/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.5216 - loss: 0.6927 - val_accuracy: 0.5660 - val_loss: 5.0077
Epoch 7/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5215 - loss: 1.7848 - val_accuracy: 0.5665 - val_loss: 0.9111
Epoch 8/30
361/361 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5216 - loss: 0.7636 - val_accuracy: 0.5663 -

c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4995 - loss: 253784.7500 - val_accuracy: 0.4785 - val_loss: 33201.4609
Epoch 2/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5134 - loss: 31086.8281 - val_accuracy: 0.5063 - val_loss: 6272.8149
Epoch 3/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4995 - loss: 4688.9824 - val_accuracy: 0.4495 - val_loss: 1778.8430
Epoch 4/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4992 - loss: 1075.2596 - val_accuracy: 0.4495 - val_loss: 1351.2803
Epoch 5/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4938 - loss: 566.9865 - val_accuracy: 0.5404 - val_loss: 826.4330
Epoch 6/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4995 - loss: 270.4940 - val_accuracy: 0.5354 - val_loss: 836.5701
Epoch 7/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4850 - loss: 171.5697 - val_accuracy: 0.5682 - val_loss: 794.9569
Epoch 8/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5226 - loss: 125.9720 - v

...

38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5110 - loss: 485962.0312 - val_accuracy: 0.5194 - val_loss: 98420.0859
Epoch 2/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5207 - loss: 69040.7031 - val_accuracy: 0.5211 - val_loss: 35173.1719
Epoch 3/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4785 - loss: 9055.0898 - val_accuracy: 0.4317 - val_loss: 6989.0586
Epoch 4/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4827 - loss: 2154.7344 - val_accuracy: 0.4165 - val_loss: 5505.3765
Epoch 5/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4844 - loss: 701.9464 - val_accuracy: 0.4115 - val_loss: 3065.6912
Epoch 6/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4798 - loss: 426.4040 - val_accuracy: 0.4165 - val_loss: 2777.6538
Epoch 7/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5211 - loss: 285.4643 - val_accuracy: 0.5936 - val_loss: 2491.1309
Epoch 8/30
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5384 - loss: 123.3391

c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4951 - loss: 41585.8320 - val_accuracy: 0.4055 - val_loss: 654.6157
Epoch 2/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4878 - loss: 572.2050 - val_accuracy: 0.5119 - val_loss: 81.2439
Epoch 3/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4935 - loss: 52.8162 - val_accuracy: 0.4091 - val_loss: 44.0367
Epoch 4/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4993 - loss: 27.9718 - val_accuracy: 0.5904 - val_loss: 34.2557
Epoch 5/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5002 - loss: 4.2371 - val_accuracy: 0.5749 - val_loss: 32.5998
Epoch 6/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5082 - loss: 1.5577 - val_accuracy: 0.5863 - val_loss: 30.4439
Epoch 7/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5105 - loss: 4.4102 - val_accuracy: 0.5863 - val_loss: 25.9631
Epoch 8/30
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5105 - loss: 1.5306 - val_accurac

c:\Users\kirut\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5144 - loss: 286876.5625 - val_accuracy: 0.4432 - val_loss: 10130.1475
Epoch 2/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5036 - loss: 4727.2681 - val_accuracy: 0.4217 - val_loss: 987.3030
Epoch 3/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4992 - loss: 753.3917 - val_accuracy: 0.4230 - val_loss: 994.5132
Epoch 4/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4992 - loss: 461.3836 - val_accuracy: 0.4268 - val_loss: 965.6371
Epoch 5/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4979 - loss: 182.4870 - val_accuracy: 0.4280 - val_loss: 163.4856
Epoch 6/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4983 - loss: 20.7908 - val_accuracy: 0.4280 - val_loss: 177.0130
Epoch 7/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4872 - loss: 3.6085 - val_accuracy: 0.5694 - val_loss: 138.9474
Epoch 8/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4964 - loss: 0.6894 - val_accuracy